# Лабораторная работа №3

In [1]:
from numpy.random import exponential, poisson
from numpy import where, sqrt, array, histogram
from scipy.stats import norm, chi2

Все расчёты вести при $\gamma = 0.05$.

In [2]:
gamma = 0.05

## Задание 1. Проверка гипотезы однородности
**A. Критерий пустых блоков.**
Генерируем две независимые выборки:
$\vec{X} = \left( X_1, \dotsc, X_n \right)$ и $\vec{Y} = \left( Y_1, \dotsc, Y_m \right)$,
причём $X_i \sim F_{ \xi } \left( u \right), \, u \geq 0$ --- экспоненциальное распределение с параметром $\alpha$,
а $Y_i \sim F_{\eta} \left( u \right) = 1 - e^{-\beta \cdot u}, \, u \geq 0$.

$n = 500, \, m = 1000, \, \alpha = 1, \, \beta = 1.2$.

Подтвердится или опровергнится гипотеза?
Критерий способный, то есть при увеличении $n$ и $m$ гипотеза должна отброситься.

$n = 5000, \, m = 10000$.

Распознает ли этот критерий, что распределение разное?

### Критерий пустых блоков
Гипотеза однородности: $H_0 \; : \; F_1 \left( \cdot \right) = F_2 \left( \cdot \right) = F \left( \cdot \right)$,
то есть эти выборки построены по одной и той же случайной величине.

Строим вариационный ряд
$-\infty <
x_{\left( 1 \right) } <
x_{\left( 2 \right)} <
x_{\left( 3 \right) } <
\dotsc < x_{\left( n \right) } <
+ \infty$,
который соответствует наблюдениям $\vec{X}$.
$U_1 = \left( -\infty, x_{\left( 1 \right)} \right),
\dotsc,
U_i = \left( x_{\left( i - 1 \right)}, x_{\left( i \right)} \right),
\dotsc,
U_{n+1} = \left( x_{\left( n \right)}, +\infty \right), \,
i = \overline{2, n}$.
Точки вариационного ряда разбивают весь промежуток на $n + 1$ часть.
Точки второго вариационного ряда попадают в эти блоки.

Важно построить $\mu_0 \left( n, m \right)$ --- количество пустых блоков, то есть блоков, которые не содержат ни одной точки (частички) из второй выборки.
$\mu_0 \left( n, m \right)$, когда $n, m$ возрастают, имеет ассимптотическое распределение.

Функция $F \left( \cdot \right)$ --- это непрерывная функция распределения.
Это исключает совпадение точек $\vec{Y}$ с точками $\vec{X}$.

Если функции распределений совпадают, то точки $\vec{Y}$ распределены в блоках равномерно.

**Теорема.**
Неизвестная функция $F \left( \cdot \right)$ --- это непрерывная функция распределения,
$n, m \to \infty $ таким образом, что
$$\frac{m}{n} \to \rho > 0$$
($\rho$ не может быть бесконечностью).

Тогда
$$\frac{\mu_0 \left( n, m \right) - a_n}{\sigma_n} \sim N \left( 0, 1 \right),$$
где
$$a_n = \frac{n}{1 + \rho}, \,
\sigma_n^2 = \frac{n \cdot \rho^2}{\left( 1 + \rho \right)^3},$$
то есть и $\sigma_n$ возрастает, и $a_n$ возрастает.

Задаётся $\gamma$ --- это ошибка первого рода, критическая область согласно этому критерию
$$\mathbb{Z}_1 =
\left\{ \left( \vec{x}, \vec{y}\right) \, : \,
\mu_0 \left( n, m \right) >
\frac{n}{1 + \rho} + \frac{\rho \sqrt{n}}{\left( 1 + \rho\right)^{\frac{3}{2}}} \cdot z_{\gamma}\right\},$$
где $z_{\gamma}$ --- это решение уравнения $0.5 - \Phi \left( z_{\gamma} \right) = \gamma$.


Этот критерий способный относительно $H_1$.

In [3]:
def empty_blocks(n, m):
    rho = float(m) / n
    x = exponential(scale=1, size=n)
    y = exponential(scale=1.2, size=m)
    x_variational_series = sorted(x)
    number_of_y_in_blocks = array([0] * (n + 1), dtype='int64')
    for element in y:
        if element < x_variational_series[0]:
            number_of_y_in_blocks[0] += 1
        elif element > x_variational_series[n - 1]:
            number_of_y_in_blocks[n] += 1
        else:
            for i in range(n):
                if i < (n - 1) and element > x_variational_series[i] and element < x_variational_series[i + 1]:
                    number_of_y_in_blocks[i + 1] += 1
    mu = len(where(number_of_y_in_blocks > 0)[0])
    print "mu:", mu
    z = norm.ppf(1 - gamma / 2.)
    critical = n / (1 + rho) + rho * sqrt(n) / (1 + rho) ** (3 / 2.) * z
    print "critical:", critical
    if mu > critical:
        print "Distributions are the same"
    else:
        print "Distributions are different"

In [4]:
empty_blocks(500, 1000)

mu: 345
critical: 183.535350825
Distributions are the same


In [5]:
empty_blocks(5000, 10000)

mu: 3344
critical: 1720.01012974
Distributions are the same


**Б.$\chi^2$-критерий.**
Генерируем $k$ серий независимых наблюдений
$\vec{X}^{ \left( i \right)} = \left( X_1^{\left( i \right)}, \dotsc, X_{n_i}^{\left( i \right) } \right), \,
i = \overline{1, k}$.

Допустим, что проверяется гипотеза о распределении Пуассона с параметром $2$.

$U_i = \left\{ i - 1 \right\}, \, i = \overline{1, r - 1}, \, U_r = \left\{ r, r + 1, \dotsc \right\}$.

Гипотеза должна быть подтверждена, когда $k = 3, \, n_1 = 200, \, n_2 = 500, \, n_3 = 800, \, r = 10$
(нужно $r$ выбрать таким, чтобы в каждый промежуток попадало не меньше пяти точек), где $r$ ---
количество промежутков.

Пусть величина $\nu$ имеет распределение Пуассона.
Оно генерируется следующим образом
$$\nu = \max \left\{ j \geq 0 \, : \, \prod \limits_{i=1}^j \omega_i \geq e^{-\lambda }\right\},$$
где $\omega_i$ --- равномерно распределённые на $\left[ 0, 1 \right]$ случайные величины.

### Критерий $\chi^2$
Общее количество наблюдений $n = n_1 + \dotsc + n_k$.

Случайная величина принимает $r$ значений: $\xi_i \in \left\{ a_1, \dotsc, a_r \right\}$,
где $i = \overline{1, k}$.

$\vec{X}^{\left( 1 \right)} \sim \xi_1\sim F_1 \left( \cdot \right),
\dotsc,
\vec{X}^{\left( k \right)} \sim \xi_k \sim F_k \left( \cdot \right).$
Распределение может быть дискретным (конечным, бесконечным), непрерывным (дискретизируем промежуток).

Гипотеза $H_0$ состоит в том, что все функции распределения
$F_1 \left( \cdot \right) = \dotsc = F_k \left( \cdot \right) = F \left( \cdot \right)$, то есть нужно проверить, что это всё выборки с одной и той же функцией распределения.
На самом деле гипотезу $H_0$ можно записать иначе.

$\xi_i \sim \left\{ p_j^{\left( i \right)} \right\}, \, j = \overline{1, r}, \, i = \overline{1, k}$.
Тогда $H_0 \, : \, p_j^{\left( i \right) } = p_j, \, j = \overline{1, r}, \, i = \overline{1, k}$,
то есть гипотеза состоит в том, что распределения не зависят от номера серии $i$.

$$\Delta \left( \vec{p}, \vec{n} \right) =
\sum \limits_{i = 1}^k \sum \limits_{j = 1}^r
\frac{\left[ \nu_j^{\left( i \right)} - n_i \cdot p_j \right]^2}{n_i \cdot p_j} =
\sum \limits_{i=1}^k \sum \limits_{j=1}^r \frac{\left[ \nu_j^{\left( i \right)}\right]^2}{n_i \cdot p_j} - n.$$

Проблема состоит в том, что вероятности $p_j$ неизвестны.

Применим метод наибольшего правдоподобия для оценки $\left\{ p_j \right\}$ при условии,
что $p_1 + \dotsc + p_r = 1$.
Составляем функцию правдоподобия по всем выборкам
$$L \left( \vec{p} \right) =
\prod \limits_{i=1}^k \prod \limits_{j=1}^r p_j^{\nu_j^{\left( i \right)}} \to \max \limits_{\vec{p}}.$$

$$L \left( \vec{p} \right) = \prod \limits_{j=1}^r p_j^{\sum \limits_{i=1}^k \nu_j^{\left( i \right)}},$$
то есть наблюдения $a_j$ суммируются по всем выборкам.
Обозначим
$$\nu_j = \sum \limits_{i=1}^k \nu_j^{\left( i \right)}.$$
Это можно переписать как
$L \left( \vec{p} \right) =
p_1^{\nu_1} \cdot \dotsc \cdot p_{r-1}^{\nu_r - 1} \cdot \left( 1 - p_1 - \cdot - p_{r-1} \right)^{\nu_r} \to
\max \limits_{\vec{p}}$,
то есть у нас фактически $r - 1$ переменная.

Логарифмируем и берём производную
$$ln \, L \left( \vec{p} \right) =
\sum \limits_{j=1}^{r-1} \nu_j \cdot ln \, p_j + \nu_r \cdot ln \left( 1 - p_1 - \dotsc - p_{r-1} \right).$$
Берём производную по $p_i$.
Будет
$$\frac{\nu_j}{p_j} - \frac{\nu_r}{1 - p_1 - \dotsc - p_{r-1}} = 0, \, j = \overline{1, r - 1}.$$
Получается
$$\frac{\nu_j}{p_j} = \frac{\nu_r}{p_r}, \, j = 1, \dotsc, r-1,$$
то есть это отношение --- константа, оно не зависит от $j$, то есть
$$c = \frac{\nu_j}{p_j} = \frac{\nu_r}{p_r}, \, j = 1, \dotsc, r - 1.$$
Тогда
$$p_j = \frac{\nu_j}{c}.$$
Тогда
$$1 = \sum \limits_{j=1}^r p_j = \frac{1}{c} \sum \limits_{j=1^r \nu_j} = \frac{n}{c},$$
то есть $c = n$.
В результате
$$p_j = \frac{\nu_j}{n} \, \left(j = \overline{1, r} \right).$$
Теперь подставляем в $\Delta$, и получается, что
$$\Delta \left(\vec{p}, \vec{n} \right) =
\sum \limits_{i=1}^k \sum \limits_{j=1}^r
\frac{\left[ \nu_j^{\left( i \right)} - \frac{n_i \cdot \nu_j}{n}\right]^2}{n_i \cdot \nu_j} \cdot n =
\left[ \sum \limits_{i=1}^k \sum \limits_{j=1}^r
\frac{\left[ \nu_j^{\left( i \right)} \right]^2}{n_i \cdot \nu_j} - 1 \right] \cdot n.$$
**Теорема.**
Если оценка для вектора $\vec{p}$ построена методом наибольшего правдоподобия, то
$$\lim \limits_{n \to \infty } P \left\{ \Delta \left( \vec{p}, \vec{n} \right) < z \; \middle| \; H_0\right\} =
T_{\left( r-1 \right) \cdot \left( k-1 \right) } \left( z \right) = 1 - \gamma.$$
Отсюда находится $z_{\gamma}$.
Критическая область $\mathbb{Z}_1$ --- это множество наблюдений (векторов):
$\mathbb{Z}_1 =
\left\{ \left( \vec{x}^{\left( 1 \right)}, \dotsc, \vec{x^{\left( k \right)}} \right) \, : \,
\Delta \left( \vec{p}, \vec{n}\right) > z_{\gamma}\right\}$.
Критерий является способным, то есть при $n \to \infty$ мы не можем ошибиться и принять неверную гипотезу.

In [6]:
n1 = 200
n2 = 500
n3 = 800
n = n1 + n2 + n3
r = 3
x1 = poisson(lam=2, size=n1)
x2 = poisson(lam=2, size=n2)
x3 = poisson(lam=2, size=n3)
nu1 = histogram(x1, bins=r)[0]
nu2 = histogram(x2, bins=r)[0]
nu3 = histogram(x3, bins=r)[0]
nu = array([sum(x) for x in zip(nu1, nu2, nu3)], dtype='float64')
delta = 0
for j in range(r):
    delta += nu1[j] ** 2 / (n1 * nu[j]) + nu2[j] ** 2 / (n2 * nu[j]) + nu3[j] ** 2 / (n3 * nu[j])
delta -= 1
delta *= n
z = chi2.ppf(1 - gamma, 2 * (r - 1))
print "Delta:", delta
print "z:", z
if delta > z:
    print "Samples have different distribution"
else:
    print "Samples have the same distribution"

Delta: 4.25833216408
z: 9.48772903678
Samples have the same distribution


## Задание 2. Проверка гипотезы независимости
**А. Критерий $\chi^2.$**
Генерируем выборку
$\left( \vec{X}, \vec{Y} \right) = \left\{ \left( X_1, Y_1\right), \dotsc, \left( X_n, Y_n \right) \right\}$,
где $X_i \sim Pois \left( 2 \right), \, Y_i \sim Pois \left( 5 \right), \, U_i$ те же, что и во втором задании,
а $V_i = \left\{ i - 1 \right\}, \, i = \overline{1, k - 1}$, а $U_k = \left\{ k, k + 1, \dotsc \right\}$.

$r = 8, \, k = 15, \ n = 1000$ ($r$ и $k$ нужно выбирать такими, чтобы ни одной ячейки не было пустой).

### $\chi^2$-критерий
Он применяется как для дискретных случайных величин, так и для непрерывных.

Если $\xi\, : \, a_1, \dotsc, a_r, \, \eta \, : \, b_1, \dotsc, b_k$, то можем применить $\chi^2$-критерий.

Если случайная величина непрерывна, то разбиваем $\mathbb{R}$ на области.

Каждая из областей должна принимать хотя бы $5$ значений.

$p_{ij} = P \left\{ \xi = a_i, \eta = b_j \right\}, \, i = \overline{1, r}, \, j = \overline{1,k}$.

$H_0 \, : \, p_{ij} = p_i^{\left( 1 \right)} \cdot p_j^{\left( 2 \right)} \,
\exists \left\{ p_i^{\left( 1 \right)} \right\}, \, \left\{p_j^{\left( 2 \right)} \right\}$.
Дискретные случайные величины будут независимыми тогда и только тогда, когда их совместная вероятность равна произведению их вероятностей.
$$\sum \limits_{i=1}^n p_i^{\left( 1 \right)} = 1, \, \sum \limits_{j=1}^k p_{j}^{\left( 2 \right) } = 1.$$
Когда применяют этот метод, строят матрицу смежности.
Она выглядит следующим образом


|$\xi / \eta$|$b_1$|$b_2$|$\dotsc$|$b_k$|Сумма|
|-----|-----|-----|--------|-----|-----|
|$a_1$|$\nu_{11}$|$\nu_{12}$|$\dotsc$|$\nu_{1k}$|$\nu_{1\cdot}$|
|$a_2$|$\nu_{21}$|$\nu_{22}$|$\dotsc$|$\nu_{2k}$|$\nu_{2\cdot}$|
|$\dotsc$|$\dotsc$|$\dotsc$|$\dotsc$|$\dotsc$|$\dotsc$|
|$a_r$|$\nu_{r1}$|$\nu_{r2}$|$\dotsc$|$\nu_{rk}$|$\nu_{r\cdot}$|
|Сумма|$\nu_{\cdot 1}$|$\nu_{\cdot 2}$|$\dotsc$|$\nu_{\cdot k}$|$n$|

$\nu_{1 \cdot }$ означает, что это сумма по второму индексу.

$\nu{\cdot 1}$ означает, что сумма по первому индексу.

$n$ --- общее количество наблюдений.

$$\sum \limits_{i=1}^r \nu_{i \cdot} = n, \, \sum \limits{j=1}^r \nu_{\cdot k} = n.$$

Составляем сумму $\Delta \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right)} \right)$,
где $\vec{p}^{\left( 1 \right) } = \left( p_1^{\left( 1 \right)}, \dotsc, p_r^{\left( 1 \right)}\right)$,
а $\vec{p}^{\left( 2 \right) } = \left( p_1^{\left( 2 \right)}, \dotsc, p_k^{\left( 2 \right)}\right)$.
$$\Delta_n \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right)} \right) =
\sum \limits_{i=1}^r \sum \limits_{j=1}^k
\frac{\left( \nu_{ij} - n \cdot p_i^{\left( 1 \right)} \cdot p_j^{\left( 2 \right)}\right)^2}{n \cdot p_i^{\left( 1 \right) } \cdot p_j^{\left( 2 \right)}}=
\sum \limits_{i=1}^r \sum \limits_{j=1}^k
\frac{\nu_{ij}^2}{n \cdot p_i^{\left( 1 \right)} \cdot p_j^{\left( 2 \right)}} - n.$$
В этой сумме неизвестны $p_i^{\left( 1 \right)}$ и $p_j^{\left( 2 \right)}$.

Находим $p_i^{\left( 1 \right)}$ и $p_j^{\left( 2 \right)}$ методом наибольшего правдоподобия.

Составляем функцию правдоподобия, логарифмируем, берём производную, находим её максимум и подставляем в $\Delta$.

Функция правдоподобия имеет вид
$$L \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right) } \right) =
c \prod \limits_{i=1}^r \prod \limits_{j=1}^k
\left[ p_i^{\left( 1 \right)} \cdot p_j^{\left( 2 \right)}\right]^{\nu_{ij}} \to
\max \limits_{\vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right)}}.$$
Можно это произведение немного упростить.
Приходим к тому, что будем использовать матрицу смежности
$$L \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right) } \right) =
c \prod \limits_{i=1}^r \prod \limits_{j=1}^k \left[ p_i^{\left( 1 \right)} \right]^{\nu_{ij}}
\prod \limits_{i=1}^r \prod \limits_{j=1}^k \left[ p_j^{\left( 2 \right)}\right]^{\nu_{ij}} =
\prod \limits_{i=1}^r \left[ p_i^{\left( 1 \right)} \right]^{\nu_{i \cdot }}
\prod \limits_{j=1}^k \left[ p_j^{\left( 2 \right)} \right]^{\nu_{\cdot j}} \cdot c \to \max.$$
Оптимальные значения достигаются, когда
$$\hat{p}_i^{\left( 1 \right)} = \frac{\nu_{i \cdot }}{n}, \, i = \overline{1, r},$$
а
$$\hat{p}_j^{\left( 2 \right)} = \frac{\nu_{\cdot j}}{n}, \, j = \overline{1, k}.$$
Нужно эти вероятности подставить в $\Delta$:
$$\Delta \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right) } \right) =
\sum \limits_{i=1}^r \sum \limits_{j=1}^k
\frac{\left( \nu_{ij} - \frac{\nu_{i \cdot } \cdot \nu_{\cdot j}}{n^2} \cdot n\right)^2}{\nu_{i\cdot } \cdot \nu_{\cdot j}} \cdot n^2 =
n \cdot
\left[ \sum \limits_{i=1}^r \sum \limits_{j=1}^k \frac{\nu_{ij}^2}{\nu_{i\cdot } \cdot \nu_{\cdot j}} - 1\right].$$
**Теорема.**
Если применяется метод наибольшего правдоподобия для оценки параметров, то
$$\lim \limits_{n \to \infty } P \left\{
\Delta_n \left( \vec{p}^{\left( 1 \right)}, \vec{p}^{\left( 2 \right) } \right) < z \; \middle| ; H_0
\right\} = T_{\left( r - 1 \right) \cdot \left( k - 1 \right)} \left( z \right) = 1 - \gamma.$$
Критическая область
$\mathbb{Z}_1 =
\left\{ \left( \vec{x}, \vec{y} \right) \, : \, \Delta_n \left( \cdot \right) > z_{\gamma }\right\}$.

Этот критерий имеет свойство способности относительно альтернативной гипотезы, что случайные величины зависимы.

In [7]:
r = 3
k = 6
n = 1000
samples = array([(poisson(lam=2), poisson(lam=5)) for i in range(n)], dtype='float64')
len_interval_x = (samples[:,0].max() - samples[:,0].min() + 1.) / r
len_interval_y = (samples[:,1].max() - samples[:,1].min() + 1.) / k
nu = array([[sum(
    (samples[:,0] < samples[:,0].min() + len_interval_x * (i + 1)) &
    (samples[:,0] >= samples[:,0].min() + len_interval_x * (i)) &
    (samples[:,1] < samples[:,1].min() + len_interval_y * (j + 1)) &
    (samples[:,1] >= samples[:,1].min() + len_interval_y * (j))
) for i in range(r)] for j in range(k)], dtype='float64')
delta = 0
for i, row in enumerate(nu):
    for j, cell in enumerate(row):
        delta += (cell ** 2) / (nu[i].sum() * nu[:,j].sum())
delta -= 1
delta *= n
z = chi2.ppf(1 - gamma, (k - 1) * (r - 1))
print "Delta:", delta
print "z:", z
if delta > z:
    print "Samples are dependent"
else:
    print "Samples are independent"

Delta: 2.85328322093
z: 18.3070380533
Samples are independent
